In [103]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,plot_confusion_matrix
import numpy as np

In [3]:
data = pd.read_csv('final_processed_csv.csv')

In [4]:
data.head()

,Unnamed: 0,url,type,length,ip_present,special_char_count,common_term_count,digit_ratio,phish_hints,second_double_slash,http(s)_present,URL_depth,abnormal_subdomain
0,0,https://www.amaozom.6rcbybq.cn/,1.0,31.0,0,7,3,0.032258,0,0,0,0,0
1,1,https://www2.amaozom.kudayin.cn/,1.0,32.0,0,7,3,0.031250,0,0,0,0,1
2,2,https://wiadomosci.wp.pl/tag/pogoda,0.0,35.0,0,7,2,0.000000,1,0,0,2,0
3,3,https://www.aparatkids.com/?utm_source=aparat-...,0.0,121.0,0,18,4,0.000000,1,0,0,0,0
4,4,http://azb3s.cf/2jo4cehk1nzc1sk66897lrszwgwssp...,1.0,139.0,0,11,2,0.201439,0,0,0,2,1


In [5]:
#dropping unnecessary columns
data.drop(['Unnamed: 0','url'],axis=1,inplace=True)

In [6]:
data.head()

,type,length,ip_present,special_char_count,common_term_count,digit_ratio,phish_hints,second_double_slash,http(s)_present,URL_depth,abnormal_subdomain
0,1.0,31.0,0,7,3,0.032258,0,0,0,0,0
1,1.0,32.0,0,7,3,0.031250,0,0,0,0,1
2,0.0,35.0,0,7,2,0.000000,1,0,0,2,0
3,0.0,121.0,0,18,4,0.000000,1,0,0,0,0
4,1.0,139.0,0,11,2,0.201439,0,0,0,2,1


In [7]:
#checking for null values
df = data[data.isna().any(axis=1)]

In [8]:
print(df)

       type  length  ip_present  special_char_count  common_term_count  \
15255   NaN     NaN           0                   0                  0   

       digit_ratio  phish_hints  second_double_slash  http(s)_present  \
15255          0.0            0                    0                0   

       URL_depth  abnormal_subdomain  
15255          1                   0  


In [9]:
#droppping the empty rows
data.drop(15255,inplace=True)

In [10]:
y = data['type']
X = data.drop(['type'],axis=1)

In [11]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=0)

In [12]:
X_train.shape,X_test.shape

((17200, 10), (4301, 10))

In [61]:
#creating a utility method for storing accuracy results for each model
model_accuracy_table = []
def store_res(model,train_acc,test_acc):
    model_accuracy_table.append({"Model ":model,"Train accuracy ":train_acc,"Test accuracy ":test_acc})

In [14]:
train = xgb.DMatrix(X_train,label=y_train)
test = xgb.DMatrix(X_test,label=y_test)

In [15]:
param = {
    'max_depth':30,
    'booster':'gbtree',
    'eta':0.2,
    'objective':'multi:softmax',
    'num_class':2
}
epochs= 60

In [16]:
model = xgb.train(param,train,epochs)
predictions_test_xgb = model.predict(test)
predictions_train_xgb = model.predict(train)

[10:08:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softmax' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [17]:
accuracy_train_xgb = accuracy_score(predictions_train_xgb,y_train)
accuracy_test_xgb = accuracy_score(predictions_test_xgb,y_test)
accuracy_train_xgb,accuracy_test_xgb

(0.9498255813953488, 0.8802604045570798)

In [62]:
store_res('XGBooster',round(accuracy_train_xgb*100,3),round(accuracy_test_xgb*100,3))

In [63]:
model_accuracy_table

[{'Model ': 'XGBooster', 'Train accuracy ': 94.983, 'Test accuracy ': 88.026}]

In [20]:
log_regress = LogisticRegression(max_iter = 100)

In [21]:
log_regress.fit(X_train,y_train)

C:\Users\desai\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [22]:
predictions_train_log_reg = log_regress.predict(X_train)
predictions_test_log_reg = log_regress.predict(X_test)

In [23]:
accuracy_train_log_reg = accuracy_score(predictions_train_log_reg,y_train)
accuracy_test_log_reg = accuracy_score(predictions_test_log_reg,y_test)

In [24]:
accuracy_train_log_reg,accuracy_test_log_reg

(0.7606395348837209, 0.7593582887700535)

In [64]:
store_res('Logistic Regression',round(accuracy_train_log_reg*100,3),round(accuracy_test_log_reg*100,2))

In [65]:
model_accuracy_table

[{'Model ': 'XGBooster', 'Train accuracy ': 94.983, 'Test accuracy ': 88.026},
 {'Model ': 'Logistic Regression',
  'Train accuracy ': 76.064,
  'Test accuracy ': 75.94}]

In [27]:
classifier = RandomForestClassifier(n_estimators = 20,criterion='entropy',max_depth=25)

In [28]:
classifier.fit(X_train,y_train)

RandomForestClassifier(criterion='entropy', max_depth=25, n_estimators=20)

In [29]:
predictions_train_rfc = classifier.predict(X_train)
predictions_test_rfc = classifier.predict(X_test)

In [30]:
accuracy_train_rfc = accuracy_score(predictions_train_rfc,y_train)
accuracy_test_rfc = accuracy_score(predictions_test_rfc,y_test)

In [31]:
accuracy_train_rfc,accuracy_test_rfc

(0.9481395348837209, 0.8832829574517554)

In [66]:
store_res('Random Forest Classifier',round(accuracy_train_rfc*100,3),round(accuracy_test_rfc*100,3))

In [33]:
tree = DecisionTreeClassifier(max_depth=45)

In [34]:
tree.fit(X_train,y_train)

DecisionTreeClassifier(max_depth=45)

In [35]:
pred_train_dt = tree.predict(X_train)
pred_test_dt = tree.predict(X_test)

In [36]:
accuracy_train_dt = accuracy_score(y_train,pred_train_dt)
accuracy_test_dt = accuracy_score(y_test,pred_test_dt)

In [37]:
accuracy_train_dt,accuracy_test_dt

(0.9500581395348837, 0.8691002092536619)

In [67]:
store_res('decision Tree',round(accuracy_train_dt*100,3),round(accuracy_test_dt*100,3))

In [68]:
model_accuracy_table

[{'Model ': 'XGBooster', 'Train accuracy ': 94.983, 'Test accuracy ': 88.026},
 {'Model ': 'Logistic Regression',
  'Train accuracy ': 76.064,
  'Test accuracy ': 75.94},
 {'Model ': 'Random Forest Classifier',
  'Train accuracy ': 94.814,
  'Test accuracy ': 88.328},
 {'Model ': 'decision Tree',
  'Train accuracy ': 95.006,
  'Test accuracy ': 86.91}]

In [40]:
svm = SVC(C=100,gamma=0.1)

In [41]:
svm.fit(X_train,y_train)

SVC(C=100, gamma=0.1)

In [42]:
preds_train_svm = svm.predict(X_train)
preds_test_svm = svm.predict(X_test)

In [43]:
svm_train_acc = accuracy_score(preds_train_svm,y_train)
svm_test_acc = accuracy_score(preds_test_svm,y_test)

In [44]:
svm_train_acc,svm_test_acc

(0.911046511627907, 0.8553824691932109)

In [69]:
store_res('SVM',round(svm_train_acc*100,3),round(svm_test_acc*100,3))

In [70]:
model_accuracy_table

[{'Model ': 'XGBooster', 'Train accuracy ': 94.983, 'Test accuracy ': 88.026},
 {'Model ': 'Logistic Regression',
  'Train accuracy ': 76.064,
  'Test accuracy ': 75.94},
 {'Model ': 'Random Forest Classifier',
  'Train accuracy ': 94.814,
  'Test accuracy ': 88.328},
 {'Model ': 'decision Tree',
  'Train accuracy ': 95.006,
  'Test accuracy ': 86.91},
 {'Model ': 'SVM', 'Train accuracy ': 91.105, 'Test accuracy ': 85.538}]

In [47]:
knn = KNeighborsClassifier(n_neighbors=1)

In [48]:
knn.fit(X_train,y_train)

KNeighborsClassifier(n_neighbors=1)

In [49]:
preds_knn_train = knn.predict(X_train)
preds_knn_test = knn.predict(X_test)

In [50]:
knn_train_acc = accuracy_score(preds_knn_train,y_train)
knn_test_acc = accuracy_score(preds_knn_test,y_test)

In [51]:
knn_train_acc,knn_test_acc

(0.9328488372093023, 0.8374796558939781)

In [52]:
def find_n(n):
    knn = KNeighborsClassifier(n_neighbors=n)
    knn.fit(X_train,y_train)
    preds_knn_train = knn.predict(X_train)
    preds_knn_test = knn.predict(X_test)
    knn_train_acc = accuracy_score(preds_knn_train,y_train)
    knn_test_acc = accuracy_score(preds_knn_test,y_test)
    return knn_train_acc,knn_test_acc

In [53]:
acc = []
for i in range(1,10):
    max_train,max_test = find_n(i)
    max_train = max_train*100
    max_test = max_test*100
    acc.append([round(max_train,1),round(max_test,1)])
    

for i in acc:
    print (i)

[93.3, 83.7]
[88.7, 82.4]
[89.6, 83.7]
[87.8, 83.2]
[88.4, 84.2]
[86.7, 83.5]
[87.1, 83.9]
[86.2, 83.4]
[86.5, 83.6]


In [71]:
store_res('KNN',round(knn_train_acc*100,3),round(knn_test_acc*100,3))

In [72]:
model_accuracy_table

[{'Model ': 'XGBooster', 'Train accuracy ': 94.983, 'Test accuracy ': 88.026},
 {'Model ': 'Logistic Regression',
  'Train accuracy ': 76.064,
  'Test accuracy ': 75.94},
 {'Model ': 'Random Forest Classifier',
  'Train accuracy ': 94.814,
  'Test accuracy ': 88.328},
 {'Model ': 'decision Tree',
  'Train accuracy ': 95.006,
  'Test accuracy ': 86.91},
 {'Model ': 'SVM', 'Train accuracy ': 91.105, 'Test accuracy ': 85.538},
 {'Model ': 'KNN', 'Train accuracy ': 93.285, 'Test accuracy ': 83.748}]

In [73]:
results_table = pd.DataFrame(model_accuracy_table)

In [74]:
results_table

,Model,Train accuracy,Test accuracy
0,XGBooster,94.983,88.026
1,Logistic Regression,76.064,75.940
2,Random Forest Classifier,94.814,88.328
3,decision Tree,95.006,86.910
4,SVM,91.105,85.538
5,KNN,93.285,83.748


###  Plotting confusion marix

In [111]:
#XGBooster
xgb_train_cm = confusion_matrix(predictions_train_xgb,y_train)
xgb_test_cm = confusion_matrix(predictions_test_xgb,y_test)

In [112]:
xgb_train_cm,xgb_test_cm

(array([[9886,  549],
        [ 314, 6451]], dtype=int64),
 array([[2347,  292],
        [ 223, 1439]], dtype=int64))

In [114]:
#Logistic Regression
log_reg_train_cm = confusion_matrix(predictions_train_log_reg,y_train)
log_reg_test_cm = confusion_matrix(predictions_test_log_reg,y_test)

In [115]:
log_reg_train_cm,log_reg_test_cm

(array([[8387, 2304],
        [1813, 4696]], dtype=int64),
 array([[2118,  583],
        [ 452, 1148]], dtype=int64))

In [116]:
#Random Forest Classifier
rfc_train_cm = confusion_matrix(predictions_train_rfc,y_train)
rfc_test_cm = confusion_matrix(predictions_test_rfc,y_test)

In [117]:
rfc_train_cm,rfc_test_cm

(array([[9876,  568],
        [ 324, 6432]], dtype=int64),
 array([[2360,  292],
        [ 210, 1439]], dtype=int64))

In [118]:
#Decision Tree
dt_train_cm = confusion_matrix(pred_train_dt,y_train)
dt_test_cm = confusion_matrix(pred_test_dt,y_test)

In [119]:
dt_train_cm,dt_test_cm

(array([[9959,  618],
        [ 241, 6382]], dtype=int64),
 array([[2336,  329],
        [ 234, 1402]], dtype=int64))

In [120]:
#SVM
svm_train_cm = confusion_matrix(preds_train_svm,y_train)
svm_test_cm = confusion_matrix(preds_test_svm,y_test)

In [121]:
svm_train_cm,svm_test_cm

(array([[9601,  931],
        [ 599, 6069]], dtype=int64),
 array([[2299,  351],
        [ 271, 1380]], dtype=int64))

In [123]:
#KNN
knn_train_cm = confusion_matrix(preds_knn_train,y_train)
knn_test_cm = confusion_matrix(preds_knn_test,y_test)

In [124]:
knn_train_cm, knn_test_cm

(array([[9585,  540],
        [ 615, 6460]], dtype=int64),
 array([[2256,  385],
        [ 314, 1346]], dtype=int64))

In [156]:
model_prec_recall_table = []
def store_prec_recall(model,train_prec,test_prec,train_recall,test_recall):
    model_prec_recall_table.append({'Model':model,"Train Precision":train_prec,"Test Precision":test_prec,"Train Recall":train_recall,"Test Recall":test_recall})

In [164]:
model_prec_recall_table

[{'Model': 'XGB',
  'Train Precision': 92.157,
  'Test Precision': 83.131,
  'Train Recall': 95.358,
  'Test Recall': 86.582},
 {'Model': 'Lgistic regression',
  'Train Precision': 67.086,
  'Test Precision': 66.32,
  'Train Recall': 72.146,
  'Test Recall': 71.75},
 {'Model': 'Random Forest Classifier',
  'Train Precision': 91.886,
  'Test Precision': 91.886,
  'Train Recall': 95.204,
  'Test Recall': 87.265},
 {'Model': 'Decision Tree',
  'Train Precision': 91.171,
  'Test Precision': 80.994,
  'Train Recall': 96.361,
  'Test Recall': 85.697},
 {'Model': 'SVM',
  'Train Precision': 86.7,
  'Test Precision': 79.723,
  'Train Recall': 91.017,
  'Test Recall': 83.586},
 {'Model': 'KNN',
  'Train Precision': 92.286,
  'Test Precision': 77.759,
  'Train Recall': 91.307,
  'Test Recall': 81.084}]

In [139]:
def precision(train_cm,test_cm):
    tr_TN,tr_FP,tr_FN,tr_TP = train_cm.ravel()
    te_TN,te_FP,te_FN,te_TP = test_cm.ravel()
    return [round(tr_TP/(tr_TP+tr_FP)*100,3),round(te_TP/(te_TP+te_FP)*100,3)]
    
def recall(train_cm,test_cm):
    tr_TN,tr_FP,tr_FN,tr_TP = train_cm.ravel()
    te_TN,te_FP,te_FN,te_TP = test_cm.ravel()
    return [round(tr_TP/(tr_TP+tr_FN)*100,3),round(te_TP/(te_TP+te_FN)*100,3)]

In [158]:
store_prec_recall("XGB",precision(xgb_train_cm,xgb_test_cm)[0],precision(xgb_train_cm,xgb_test_cm)[1],recall(xgb_train_cm,xgb_test_cm)[0],recall(xgb_train_cm,xgb_test_cm)[1])

In [159]:
store_prec_recall("Lgistic regression",precision(log_reg_train_cm,log_reg_test_cm)[0],precision(log_reg_train_cm,log_reg_test_cm)[1],recall(log_reg_train_cm,log_reg_test_cm)[0],recall(log_reg_train_cm,log_reg_test_cm)[1])

In [160]:
store_prec_recall("Random Forest Classifier",precision(rfc_train_cm,rfc_test_cm)[0],precision(rfc_train_cm,rfc_test_cm)[0],recall(rfc_train_cm,rfc_test_cm)[0],recall(rfc_train_cm,rfc_test_cm)[1])

In [161]:
store_prec_recall("Decision Tree",precision(dt_train_cm,dt_test_cm)[0],precision(dt_train_cm,dt_test_cm)[1],recall(dt_train_cm,dt_test_cm)[0],recall(dt_train_cm,dt_test_cm)[1])

In [162]:
store_prec_recall("SVM",precision(svm_train_cm,svm_test_cm)[0],precision(svm_train_cm,svm_test_cm)[1],recall(svm_train_cm,svm_test_cm)[0],recall(svm_train_cm,svm_test_cm)[1])

In [163]:
store_prec_recall("KNN",precision(knn_train_cm, knn_test_cm)[0],precision(knn_train_cm, knn_test_cm)[1],recall(knn_train_cm, knn_test_cm)[0],recall(knn_train_cm, knn_test_cm)[1])

In [195]:
pre_rec_df = pd.DataFrame(model_prec_recall_table)

### Finding F1 score

In [167]:
def f1_score(prec,recall):
    return 2*(prec*recall)/(prec+recall)

93.73017844972402


In [184]:
model_f1_table = []
def f1_score(model,prec,recall):
    model_f1_table.append({"Model":model,"F1 score Train":2*(prec[0]*recall[0])/(prec[0]+recall[0]),"F1 score Test":2*(prec[1]*recall[1])/(prec[1]+recall[1])}) 

In [185]:
f1_score("XGB",precision(xgb_train_cm,xgb_test_cm),recall(xgb_train_cm,xgb_test_cm))

In [186]:
f1_score("Lgistic regression",precision(log_reg_train_cm,log_reg_test_cm),recall(log_reg_train_cm,log_reg_test_cm))

In [187]:
f1_score("Random Forest Classifier",precision(rfc_train_cm,rfc_test_cm),recall(rfc_train_cm,rfc_test_cm))

In [188]:
f1_score("Decision Tree",precision(dt_train_cm,dt_test_cm),recall(dt_train_cm,dt_test_cm))

In [189]:
f1_score("SVM",precision(svm_train_cm,svm_test_cm),recall(svm_train_cm,svm_test_cm))

In [190]:
f1_score("KNN",precision(knn_train_cm, knn_test_cm),recall(knn_train_cm, knn_test_cm))

In [191]:
model_f1_table

[{'Model': 'XGB',
  'F1 score Train': 93.73017844972402,
  'F1 score Test': 84.82141311508252},
 {'Model': 'Lgistic regression',
  'F1 score Train': 69.52405418294644,
  'F1 score Test': 68.92822481350039},
 {'Model': 'Random Forest Classifier',
  'F1 score Train': 93.51557799989311,
  'F1 score Test': 85.14785223831545},
 {'Model': 'Decision Tree',
  'F1 score Train': 93.69418265682658,
  'F1 score Test': 83.27915505936134},
 {'Model': 'SVM',
  'F1 score Train': 88.80606694913824,
  'F1 score Test': 81.60881124738992},
 {'Model': 'KNN',
  'F1 score Train': 91.79388976703905,
  'F1 score Test': 79.38669952091058}]

In [194]:
f1_df = pd.DataFrame(model_f1_table)

In [196]:
results_table

,Model,Train accuracy,Test accuracy
0,XGBooster,94.983,88.026
1,Logistic Regression,76.064,75.940
2,Random Forest Classifier,94.814,88.328
3,decision Tree,95.006,86.910
4,SVM,91.105,85.538
5,KNN,93.285,83.748


In [197]:
pre_rec_df

,Model,Train Precision,Test Precision,Train Recall,Test Recall
0,XGB,92.157,83.131,95.358,86.582
1,Lgistic regression,67.086,66.320,72.146,71.750
2,Random Forest Classifier,91.886,91.886,95.204,87.265
3,Decision Tree,91.171,80.994,96.361,85.697
4,SVM,86.700,79.723,91.017,83.586
5,KNN,92.286,77.759,91.307,81.084


In [198]:
f1_df

,Model,F1 score Train,F1 score Test
0,XGB,93.730178,84.821413
1,Lgistic regression,69.524054,68.928225
2,Random Forest Classifier,93.515578,85.147852
3,Decision Tree,93.694183,83.279155
4,SVM,88.806067,81.608811
5,KNN,91.793890,79.386700
